In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.8 MB/s eta 0:00:0000:0100:01
  Attempting unins

In [49]:
import numpy as np
import pandas as pd
from ultralytics import YOLO

In [50]:
images = "/kaggle/input/jaundice-image-data"

In [51]:
yolo_model = YOLO('yolov8n.pt')

In [52]:
results = yolo_model("/kaggle/input/chicken-disease-dataset/chicken_disease/train/Coccidiosis/cocci.1.jpg")


image 1/1 /kaggle/input/chicken-disease-dataset/chicken_disease/train/Coccidiosis/cocci.1.jpg: 640x640 1 pizza, 8.3ms
Speed: 11.7ms preprocess, 8.3ms inference, 87.6ms postprocess per image at shape (1, 3, 640, 640)


In [53]:
for r in results:
    boxes = r.boxes

    for box in boxes:
        x1,y1,x2,y2 = box.xyxy[0]
        conf_score = box.conf[0]

In [54]:
print(x1,y1,x2,y2)

tensor(11.8066, device='cuda:0') tensor(4.9857, device='cuda:0') tensor(222.4064, device='cuda:0') tensor(223.2017, device='cuda:0')


In [55]:
import cv2

img = cv2.imread(f"/kaggle/input/chicken-disease-dataset/chicken_disease/train/Coccidiosis/cocci.1.jpg")

In [56]:
img

array([[[  3,  38,  52],
        [  0,  27,  44],
        [  0,  21,  41],
        ...,
        [ 36,  49,  71],
        [ 43,  54,  74],
        [ 51,  62,  82]],

       [[  3,  34,  49],
        [  0,  26,  43],
        [  0,  19,  40],
        ...,
        [ 37,  50,  72],
        [ 39,  53,  72],
        [ 42,  56,  75]],

       [[  3,  28,  44],
        [  3,  26,  42],
        [  6,  23,  42],
        ...,
        [ 38,  55,  76],
        [ 37,  54,  75],
        [ 40,  57,  78]],

       ...,

       [[ 33,  70,  98],
        [ 30,  65,  91],
        [ 32,  61,  88],
        ...,
        [ 95, 136, 168],
        [100, 141, 174],
        [101, 142, 175]],

       [[ 19,  49,  78],
        [ 16,  45,  72],
        [ 22,  46,  74],
        ...,
        [ 69, 110, 142],
        [ 74, 117, 150],
        [ 89, 132, 165]],

       [[ 10,  38,  68],
        [ 17,  44,  71],
        [ 16,  38,  66],
        ...,
        [ 62, 103, 135],
        [ 67, 110, 143],
        [ 80, 125, 158]]

In [57]:
x1,y1,x2,y2 = int(x1.item()), int(y1.item()), int(x2.item()), int(y2.item())

img_cropped = img[y1:y2, x1:x2]

In [ ]:
import matplotlib.pyplot as plt
import cv2

# BGR → RGB conversion for correct colors

plt.imshow(cv2.cvtColor(img_cropped, cv2.COLOR_BGR2RGB))
plt.axis('off')  # hide axis
plt.show()

In [59]:
from torchvision import transforms
from PIL import Image
import torch
import os

# Data Augmentation Pipeline
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),                 # Resize to match model input
    transforms.RandomHorizontalFlip(p=0.5),        # Flip image horizontally
    transforms.RandomRotation(degrees=15),         # Rotate ±15 degrees
    transforms.ColorJitter(brightness=0.3,         # Randomly change brightness
                            contrast=0.3,
                            saturation=0.2,
                            hue=0.1),
    transforms.ToTensor(),                         # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],   # Normalization (ImageNet)
                         std=[0.229, 0.224, 0.225])
])

"""# Example: Loading and augmenting one image
img_path = "/kaggle/input/jaundice-image-data/normal/normal (1).jpg"
img = Image.open(img_path).convert("RGB")


img_tensor = train_transforms(img)

print(img_tensor.shape)  # torch.Size([3, 224, 224])"""

'# Example: Loading and augmenting one image\nimg_path = "/kaggle/input/jaundice-image-data/normal/normal (1).jpg"\nimg = Image.open(img_path).convert("RGB")\n\n\nimg_tensor = train_transforms(img)\n\nprint(img_tensor.shape)  # torch.Size([3, 224, 224])'

In [60]:
import os
img_dir = "/kaggle/input/chicken-disease-dataset/chicken_disease/train"

image_data = []
label_data = []

for label_idx, classes in enumerate(sorted(os.listdir(img_dir))):
    
    class_folder = f'{img_dir}/{classes}'
    
    for image in os.listdir(class_folder):
        
        img_path = f'{class_folder}/{image}'
        
        img = Image.open(img_path).convert("RGB")
        ""
        img_tensor = train_transforms(img)
        
        image_data.append(img_tensor)
        
        label_data.append(label_idx)

In [61]:
image_data[76]

tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -1

In [62]:
len(image_data)

6508

In [63]:
import torch 
import torch.nn as nn
import torchvision.models as models

In [64]:
resnet = models.resnext50_32x4d(pretrained = True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 197MB/s]


In [65]:
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 4)  # change final layer

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet.to(device)

In [105]:
device

device(type='cuda')

In [106]:
sorted(os.listdir(img_dir))

['Coccidiosis', 'Healthy', 'New Castle Disease', 'Salmonella']

In [107]:
from torchvision import transforms
from PIL import Image
import torch
import os

# Data Augmentation Pipeline
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),                 # Resize to match model input

    transforms.ToTensor(),                         # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],   # Normalization (ImageNet)
                         std=[0.229, 0.224, 0.225])
])


import os
img_dir = f"/kaggle/input/chicken-disease-dataset/chicken_disease/validation"

valid_image_data = []
valid_label_data = []

for label_idx, classes in enumerate(sorted(os.listdir(img_dir))):
    
    class_folder = f'{img_dir}/{classes}'
    
    for image in os.listdir(class_folder):
        
        img_path = f'{class_folder}/{image}'
        
        img = Image.open(img_path).convert("RGB")
        
        img_tensor = test_transforms(img)
        
        valid_image_data.append(img_tensor)
        
        valid_label_data.append(label_idx)

In [113]:
import random

datas = list(zip(image_data, label_data))
random.shuffle(datas)

valid = list(zip(valid_image_data, valid_label_data))
random.shuffle(valid)

In [114]:
train_img, train_label = zip(*datas)

valid_img, valid_label = zip(*valid)

In [115]:
from torch.utils.data import DataLoader, Dataset


class MyCustomDataset(Dataset):

    def __init__(self, img, lbl):
        self.img = img
        self.lbl = torch.tensor(lbl, dtype = torch.float32)

    def __len__(self):
        return len(self.lbl)

    def __getitem__(self, idx):
        return self.img[idx], self.lbl[idx]

In [116]:
train_dataset = MyCustomDataset(train_img, train_label)
valid_dataset = MyCustomDataset(valid_img, valid_label)

In [117]:
train_data = DataLoader(train_dataset,shuffle = True, batch_size = 32)
valid_data  = DataLoader(valid_dataset, batch_size = 32)

In [121]:
epochs = 10
lr = 1e-3
optimizers = torch.optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()

In [122]:
import torch.nn.functional as F

In [123]:
for epoch in range(epochs):

    total_loss = 0
    model.train()
    accuracy = 0
    total_size = 0

    for batch in train_data:
        
        images, actual = batch

        images, actual = images.to(device), actual.to(device)

        optimizers.zero_grad()
        
        logits = model(images)
        
        loss = criterion(logits, actual.long())

        loss.backward()

        optimizers.step()

        total_loss += loss.item()

    #----------------------------------------------------------------------------------------------#

    model.eval()
    with torch.no_grad():
        
        for batch in valid_data:
            
            img, actual = batch
            img, actual = img.to(device), actual.to(device)
            
            output = model(img)
            
            index = torch.argmax(F.softmax(output, dim = -1), dim = -1)

            accuracy += (index == actual).sum().item()
            total_size += img.size(0)
            
    if total_size != 0:
        print(f"--------------------------------------------------------")
        
        print(f"Epochs: {epoch+1} | Loss: {total_loss:.3f} | Accuracy: {(accuracy/total_size) * 100:.2f}")

    else:
        break

---------------------------------------------------------------------
Epochs: 1 | Loss: 73.83372344076633 | Accuracy: 91.39
---------------------------------------------------------------------
Epochs: 2 | Loss: 54.17342484742403 | Accuracy: 92.67
---------------------------------------------------------------------
Epochs: 3 | Loss: 41.362869603559375 | Accuracy: 94.60
---------------------------------------------------------------------
Epochs: 4 | Loss: 35.72265567071736 | Accuracy: 94.99
---------------------------------------------------------------------
Epochs: 5 | Loss: 33.928777230903506 | Accuracy: 92.80
---------------------------------------------------------------------
Epochs: 6 | Loss: 28.055554816499352 | Accuracy: 93.96
---------------------------------------------------------------------
Epochs: 7 | Loss: 27.202898933086544 | Accuracy: 94.34
---------------------------------------------------------------------
Epochs: 8 | Loss: 20.047927118837833 | Accuracy: 95.50
---

In [126]:
torch.save(model.state_dict(), "saved_model.pth")

In [128]:
model.load_state_dict(torch.load(f"/kaggle/working/saved_model.pth"))

<All keys matched successfully>

In [129]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Prams: {total_params}")

Total Prams: 22988100


In [159]:
img_dir = f"/kaggle/input/chicken-disease-dataset/chicken_disease/test"

test_image_data = []
test_label_data = []

for label_idx, classes in enumerate(sorted(os.listdir(img_dir))):
    
    class_folder = f'{img_dir}/{classes}'
    
    for image in os.listdir(class_folder):
        
        img_path = f'{class_folder}/{image}'
        
        img = Image.open(img_path).convert("RGB")
        
        img_tensor = test_transforms(img)
        
        test_image_data.append(img_tensor)
        
        test_label_data.append(label_idx)

test_dataset = MyCustomDataset(test_image_data, test_label_data)

In [166]:
assert len(test_image_data) == len(test_label_data)
print(f"same chha")

same chha


In [167]:
test_loader = DataLoader(test_dataset, batch_size = len(test_image_data))

In [168]:
for img, lbl in test_loader:
    print(len(img), len(lbl))
    
    break

781 781


In [169]:
model.eval()

with torch.no_grad():

    for batch in test_loader:

        img, lbl = batch
        img, lbl = img.to(device), lbl.to(device)
        
        prediction = model(img)

        index = torch.argmax(F.softmax(prediction, dim = -1), dim = -1)

        accuracy = (index == lbl).sum().item()

        total_size = img.size(0)

    accuracy_score = (accuracy/total_size) * 100
    print(f"The accuracy is {accuracy_score:.2f}%")

The accuracy is 95.13%


In [220]:
def inference(img_path):

    labels = sorted(os.listdir("/kaggle/input/chicken-disease-dataset/chicken_disease/train"))

    img = Image.open(img_path).convert("RGB")
            
    img_tensor = train_transforms(img).unsqueeze(0).to(device)
    
    model.eval()
    
    with torch.no_grad():
    
        logits = model(img_tensor)
        index = torch.argmax(F.softmax(logits, dim = -1), dim = -1)
        confidence_score = F.softmax(logits, dim = -1)
    
        if index.item() == 0:
            print(f'The class is: {labels[index.item()]} with conf score: {confidence_score[0][index].item() * 100 :.2f}%')

        if index.item() == 1:
            print(f'The class is: {labels[index.item()]} with conf score: {confidence_score[0][index].item() * 100 :.2f}%')

        if index.item() == 2:
            print(f'The class is: {labels[index.item()]} with conf score: {confidence_score[0][index].item() * 100 :.2f}%')

        if index.item() == 3:
            print(f'The class is: {labels[index.item()]} with conf score: {confidence_score[0][index].item() * 100 :.2f}%')


In [223]:
inference(
"/kaggle/input/chicken-disease-dataset/chicken_disease/test/New Castle Disease/pcrncd.71.jpg")

The class is: New Castle Disease with conf score: 97.08%


In [153]:
sorted(os.listdir("/kaggle/input/chicken-disease-dataset/chicken_disease/train"))

['Coccidiosis', 'Healthy', 'New Castle Disease', 'Salmonella']